In this notebook we test what happens if a user provides liquidity for their own DCA, using the assets that they are trading.

In [1]:
import sys
sys.path.append('../../..')

from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.amm.global_state import GlobalState
from hydradx.model.amm.trade_strategies import dca_with_lping, omnipool_arbitrage
from hydradx.model.run import run
from mpmath import mpf, mp

mp.dps = 50


In [2]:
prices = {'DOT': 9, 'HDX': 0.02, 'USDT': 1, 'WETH': 3500, 'iBTC': 70000}

assets = {
    'DOT': {'usd price': prices['DOT'], 'weight': mpf(0.40)},
    'HDX': {'usd price': prices['HDX'], 'weight': mpf(0.10)},
    'USDT': {'usd price': prices['USDT'], 'weight': mpf(0.30)},
    'WETH': {'usd price': prices['WETH'], 'weight': mpf(0.10)},
    'iBTC': {'usd price': prices['iBTC'], 'weight': mpf(0.10)}
}

lrna_price_usd = 35
initial_omnipool_tvl = 20000000
liquidity = {}
lrna = {}

for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd

init_pool = OmnipoolState(
    tokens={tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets},
    lrna_fee=0.0005,
    asset_fee=0.0025,
    withdrawal_fee=True,
    preferred_stablecoin='USDT'
)


In [3]:

sell_size = 0.1  # total amount sold as percentage of Omnipool liquidity
tkn_sell = 'HDX'
tkn_buy = 'USDT'
trade_ct = 10000  # number of pieces trade is broken up into

total_sell_amt = sell_size * init_pool.liquidity[tkn_sell]
agent = Agent(holdings = {"HDX": mpf(total_sell_amt)})

omnipool1 = init_pool.copy()
omnipool2 = init_pool.copy()
agent1 = agent.copy()
agent2 = agent.copy()

arb_agent = Agent(
    holdings={tkn: mpf(omnipool1.liquidity[tkn])*10 for tkn in omnipool1.liquidity},
    unique_id='arb_agent',
    trade_strategy=omnipool_arbitrage('omnipool')
)


# add DOT liquidity
omnipool1.add_liquidity(agent1, agent1.holdings[tkn_sell], tkn_sell)

shares_amt_per_loop = agent1.holdings[('omnipool', tkn_sell)] / trade_ct

agent1.trade_strategy = dca_with_lping('omnipool', tkn_sell, tkn_buy, shares_amt_per_loop)

state = GlobalState(
    agents={'arb_agent': arb_agent, 'agent': agent1},
    pools={'omnipool': omnipool1},
    external_market=prices
)

events = run(state, trade_ct*2)

agent1final = events[-1].agents['agent']
omnipool1final = events[-1].pools['omnipool']

# remove USDT liquidity
omnipool1final.remove_liquidity(agent1final, agent1final.holdings[('omnipool', tkn_buy)], tkn_buy)

# sell LRNA for USDT
omnipool1final.swap(agent1final, tkn_sell='LRNA', tkn_buy=tkn_buy, sell_quantity=agent1final.holdings['LRNA'])


Starting simulation...
Execution time: 33.164 seconds.


Omnipool: omnipool
********************************
tvl cap: inf
LRNA imbalance: -181.70245513990755915368158798683578032478461250475
lrna fee:

    DOT: 0.05%
    HDX: 0.05%
    USDT: 0.05%
    WETH: 0.05%
    iBTC: 0.05%

asset fee:

    DOT: 0.25%
    HDX: 0.25%
    USDT: 0.25%
    WETH: 0.25%
    iBTC: 0.25%

asset pools: (

    *DOT*
    asset quantity: 888888.888888889
    lrna quantity: 228571.42857142858
    USD price: 8.984768886735
    weight: 228571.42857142858/571625.3718977205 (0.39986228710003147)
    weight cap: 1
    total shares: 888888.8888888889382321344277847351299391852484809
    protocol shares: 888888.8888888889382321344277847351299391852484809

    *HDX*
    asset quantity: 100144306.8826822
    lrna quantity: 57095.52172418676
    USD price: 0.019920866541
    weight: 57095.52172418676/571625.3718977205 (0.09988276331163747)
    weight cap: 1
    total shares: 100011274.11167736800737974687374039284328742472165
    protocol shares: 100011274.1116773680073797468

In [4]:
omnipool2 = init_pool.copy()
agent2 = agent.copy()

arb_agent2 = Agent(
    holdings={tkn: mpf(omnipool1.liquidity[tkn])*10 for tkn in omnipool1.liquidity},
    unique_id='arb_agent',
    trade_strategy=omnipool_arbitrage('omnipool')
)

sell_amt_per_loop = agent1.holdings[tkn_sell] / trade_ct

for i in range(trade_ct):
    omnipool2.swap(agent2, tkn_sell=tkn_sell, tkn_buy=tkn_buy, sell_quantity=sell_amt_per_loop)  # agent2 is only doing the trading
    omnipool2.swap(arb_agent2, tkn_sell=tkn_buy, tkn_buy=tkn_sell, buy_quantity=sell_amt_per_loop*(1-0.003))  # arb_agent2 is undoing the swap

# agent 2 sells remaining DOT for USDT
omnipool2.swap(agent2, tkn_sell=tkn_sell, tkn_buy=tkn_buy, sell_quantity=agent2.holdings[tkn_sell])


Omnipool: omnipool
********************************
tvl cap: inf
LRNA imbalance: 0.0
lrna fee:

    DOT: 0.05%
    HDX: 0.05%
    USDT: 0.05%
    WETH: 0.05%
    iBTC: 0.05%

asset fee:

    DOT: 0.25%
    HDX: 0.25%
    USDT: 0.25%
    WETH: 0.25%
    iBTC: 0.25%

asset pools: (

    *DOT*
    asset quantity: 888888.888888889
    lrna quantity: 228571.42857142858
    USD price: 8.479265070625
    weight: 228571.42857142858/571428.5714285715 (0.39999999999999997)
    weight cap: 1
    total shares: 888888.8888888889382321344277847351299391852484809
    protocol shares: 888888.8888888889382321344277847351299391852484809

    *HDX*
    asset quantity: 110000000.0
    lrna quantity: 51950.64935064935
    USD price: 0.015573349924
    weight: 51950.64935064935/571428.5714285715 (0.09091363636363635)
    weight cap: 1
    total shares: 100000000.00000000346944695195361411660147604826247
    protocol shares: 100000000.00000000346944695195361411660147604826247

    *USDT*
    asset quantity: 

In [5]:
from pprint import pprint

# pprint(omnipool1)
# pprint(omnipool2)
pprint(agent1final.holdings)
pprint(agent2.holdings)

{'DOT': 0,
 'HDX': mpf('0.0'),
 'LRNA': mpf('0.0'),
 'USD': 0,
 'USDT': mpf('198736.42792629632228757784131711667874584961904403873'),
 'WETH': 0,
 'iBTC': 0,
 ('omnipool', 'HDX'): mpf('0.0'),
 ('omnipool', 'USDT'): mpf('0.0')}
{'HDX': mpf('0.0'),
 'USDT': mpf('175943.98447035989494774259231601746620818571898243519')}


In [6]:
lrna_prices = [event.pools['omnipool'].price(event.pools['omnipool'], 'LRNA', 'USDT') for event in events]
pprint([lrna_prices[i] for i in range(0, len(lrna_prices), 1000)])

[mpf('34.999767101427057872724108941317573159231474194282939'),
 mpf('34.992796034641425075876872043256281793600147915671942'),
 mpf('34.984285627511737173636095265984938870538273842854602'),
 mpf('34.974689883735910446095423604252570414097167898860359'),
 mpf('34.963794354282794423939987979366632384358473602896618'),
 mpf('34.951844707106313051515159083065088650079324934114366'),
 mpf('34.992138042197358459436830991200976708409242210298726'),
 mpf('34.97741157808571694776393232266531343480016151157854'),
 mpf('34.961676632497661347163896288307873005741729666213346'),
 mpf('34.996387974402178383324684369324145517624236456338492'),
 mpf('34.977736045337552705140781570401782497749304509347791'),
 mpf('34.977736045337552705140781570401782497749304509347791'),
 mpf('34.977736045337552705140781570401782497749304509347791'),
 mpf('34.977736045337552705140781570401782497749304509347791'),
 mpf('34.977736045337552705140781570401782497749304509347791'),
 mpf('34.97773604533755270514078157040178